<h1><center>Laboratorio 5: Benchmark Bayesiano 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nelson Moreno Cabañas


### **Link de repositorio de GitHub:** `https://github.com/Bruno-Moreno/LabProgra`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Objetivos principales del laboratorio](#Objetivos-principales-del-laboratorio)

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo bayesiano para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [1]:
%%capture
!pip install praw
!pip install line_profiler
!pip install memory_profiler
import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_6o2td1zc54671.jpg?width=4000&format=pjpg&s=600bcf8560dff264f1cc7c0785ba5f6d529e0c28" width="10000">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema Bayesiano, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a las subfunciones ser ejecutadas (por linea de código, vean las clases). Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [X] Estudiar la función propuesta por el equipo docente.
- [X] Estudiar los tiempos de ejecución del Código a través de un perfilador.

In [21]:
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
    reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)
        
    subreddit  = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit = n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions),50)):
        top_n_submissions = top_submissions[:top_n]
        
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [14]:
#### Código Aquí ####
%load_ext line_profiler
%load_ext memory_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [15]:
%lprun -f praw_reddit praw_reddit() #Por alguna razón no se imprime y lo muestra aparte en mi jupypter

In [5]:
%memit praw_reddit()

peak memory: 147.60 MiB, increment: 10.98 MiB


**Respuesta:**

> Según el perfilador, el proceso que más toma es la linea top_submissions pues el método subreddit.hot recolecta los comentarios con más votaciones (entre todos los comentarios) y posiblemente los ordene lo que evidentemente toma un gran tiempo, la forma de optimizar esto vendría siendo una estructura de datos predefinida en reddit que facilite este proceso o bien una implementación en algún lenguaje más rápido.
La siguiente linea que toma bastante tiempo es praw.reddit() que accede al usuario de reddit correspondiente pero a priori no se conocen forma de optimizar este proceso salvo que tengamos acceso al método en particular. 
Por último la linea ups = ... es otra que toma bastante tiempo pero corresponden simplemente a operaciones matemáticas.

> Se puede observar además la cantidad de memoria utilizada y los incrementos en la linea %memit, vemos que la cantidad de memoria peak es bastante alta pues en algún momento se deben guardar todos los comentarios del subreddit.

## 1.2 Análisis de Tiempo con Cache

Sin duda, un factor clave en la mejora del tiempo de ejecución de una aplicación es el uso eficiente de la memoria. Por lo que es importante, que ustedes respondan las siguientes preguntas:
1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [X] Responder preguntas.
- [X] Mejorar el código con cache.


**Respuestas Teóricas:**

> 1. La memoria cache es la que permite almacenar en el disco o RAM los resultados de procesos anteriores con el fin de ser utilizados más adelante, las siglas LRU vienen de Least Recently Used y quiere decir que se almacenaran en cache los procesos por orden de ejecución.
> 2. El problema de las técnicas de Caching es que aumentamos el uso de memoria, lo cual puede traer problemas pues la memoria de nuestro hardware es limitada, o bien, de lento acceso.
> 3. Como vimos en la memoria utilizada, su uso ya es de 147MB y utilizar técnicas de caching podría aumentar considerablemente el uso de memoria.

In [23]:
#### Código para optimizar la función praw ####
@lru_cache
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
    reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)
        
    subreddit  = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit = n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions),50)):
        top_n_submissions = top_submissions[:top_n]
        
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [24]:
%%timeit
praw_reddit()

The slowest run took 7.50 times longer than the fastest. This could mean that an intermediate result is being cached.
386 ns ± 455 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
praw_reddit()

57.6 ns ± 0.138 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


> Vemos claramente la diferencia en los tiempos de ejecución al guardarlos en la memoria.

## 1.3 Obtención de Mean Posterior y Standard Error

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia Bayesiana. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$
$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [58]:
votes, post, url = praw_reddit()
votos = votes[18]
posteo = votes[18]
np.sum(votos[:, 0]), np.sum(votos[:, 1])

(110391, 6085)

In [97]:
import math 
def intervalos(votos_positivos, votos_negativos):
    suma_positivos = 0
    suma_negativos = 0
    for i in range(len(votos_positivos)):
        suma_positivos += votos_positivos[i]
        suma_negativos += votos_negativos[i]
    a = int(1+suma_positivos) #para que sean considerados long
    b = int(1+suma_negativos) #para que sean considerados long
    mu = a/(a+b)
    sigma = 1.65 * math.sqrt(a*b/(((a+b)**2)*(a+b+1)))
    return mu,sigma
    
    

def intervalos_numpy(votos_positivos, votos_negativos):
    suma_positivos = np.sum(votos_positivos)
    suma_negativos = np.sum(votos_negativos)
    a = int(1+suma_positivos) #para que sean considerados long
    b = int(1+suma_negativos) #para que sean considerados long
    mu = a/(a+b)
    sigma = 1.65 * math.sqrt(a*b/(((a+b)**2)*(a+b+1)))
    return mu,sigma


In [98]:
intervalos(votos[:, 0], votos[:, 1]) == intervalos_numpy(votos[:, 0], votos[:, 1])

True

## 1.4 Comparación de rendimiento

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. Grafique estos tiempos y observe comente los desempeños.

In [ ]:
###### Código Aquí ######

def intevalos_JIT(votos_positivos, votos negativos):
  pass 

def intevalos_numpy_JIT(votos_positivos, votos negativos):
  pass


## 1.5 Plot de Resultados Bayesianos

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [101]:
#votos = votes[len(votes)-1]  esta no funciona
#posteo = post[len(post)-1] esta no funciona
votos = votes[18]
posteo = post[18]
print("lower bounds aproximados:")
posterior_mean, std_err = intervalos_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {'Votos (+)':votos[order[:20], 0], 'Votos (-)':votos[order[:20], 1] ,'Post':np.array(posteo)[order[:20]], 'url': np.array(url[18])[order[:20]]}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



In [102]:
r_order = order[::-1][-20:]
data_dic = {'mean':posterior_mean[r_order], 'std_err':std_err[r_order], 'post':ordered_post[::-1]}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post",
                 error_x="std_err")
fig.show()

TypeError: 'float' object is not subscriptable

**Respuesta:**

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>